# HiFi-GAN 학습

## 1. 할당된 GPU 확인

In [1]:
import os
GPU_NAME = os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()
os.environ['GPU_NAME'] = GPU_NAME
print(f'GPU: {GPU_NAME}')

GPU: GeForce RTX 2080 Ti
GeForce RTX 2080 Ti
GeForce RTX 2080 Ti
GeForce RTX 2080 Ti


## 2. 필수 라이브러리 및 함수 불러오기

실행에 필요한 라이브러리 및 함수를 불러옵니다.

이 과정은 약 10분 정도 소요될 수 있습니다.

In [2]:
import sys
from pathlib import Path

## 3. 학습할 데이터셋 불러오기

학습에 사용할 음성 데이터를 업로드합니다.

## 4. 사전 학습 데이터 불러오기

In [7]:
%cd TTS
!mkdir -p "./hifigan-v2"
if not Path("./hifigan-v2/config.json").exists():
    !gdown --id 1vRxp1RH-U7gSzWgyxnKY4h_7pB3tjPmU -O hifigan-v2.zip
    !unzip -q hifigan-v2.zip -d ./
    !cp -R ./hifigan-v2/* "./hifigan-v2/"

Downloading...
From: https://drive.google.com/uc?id=1vRxp1RH-U7gSzWgyxnKY4h_7pB3tjPmU
To: /home/gpuadmin/yeonsik/TTS/TTS/hifigan-v2.zip
100%|████████████████████████████████████████| 802M/802M [01:38<00:00, 8.16MB/s]
cp: './hifigan-v2/config.json' and './hifigan-v2/config.json' are the same file
cp: './hifigan-v2/hifigan.onnx' and './hifigan-v2/hifigan.onnx' are the same file
cp: './hifigan-v2/model_file.pth.tar' and './hifigan-v2/model_file.pth.tar' are the same file
cp: './hifigan-v2/scale_stats.npy' and './hifigan-v2/scale_stats.npy' are the same file


In [13]:
if not Path("./hifigan-v2/scale_stats_new.npy").exists():
    !python bin/compute_statistics.py "./hifigan-v2/config.json" "./hifigan-v2/scale_stats_new.npy" --data_path "../filelists/wavs/"

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.1
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > There are 100 files.
100%|█████████████████████████████████████████| 100/100 [00:02<00:00, 39.30it/s]
 > Avg mel spec mean: -58.844200134277344
 > Avg mel spec scale: 15.84532356262207
 > Avg linear spec mean: -37.963314056396484
 > Avg lienar spec scale: 15.612671852111816
 > stats saved to ./hifigan-v2/scale_stats_new.npy


## 5. HiFi-GAN 학습 진행

In [8]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [12]:
# !(python TTS/bin/train_vocoder_gan.py \
#     --continue_path "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/CONTINUE_DIRECTORY")
!(CUDA_VISIBLE_DEVICES=1 nohup python TTS/bin/train_vocoder_gan.py \
    --config_path "./TTS/hifigan-v2/config.json" \
    --coqpit.data_path "./500samples_30m/wavs"  \
    --coqpit.audio.stats_path "./TTS/hifigan-v2/scale_stats_new.npy"  \
    --coqpit.output_path "./TTS/hifigan-v2/"  \
    --coqpit.num_loader_workers 2  \
    --coqpit.num_val_loader_workers 2  \
    --restore_path "./TTS/hifigan-v2/model_file.pth.tar" &> hifigan_1h.out)

 > Using CUDA:  True
 > Number of GPUs:  1
 > Git Hash: 3aa165a
 > Experiment folder: ./TTS/hifigan-v2/hifigan-v2-December-17-2021_07+35PM-3aa165a
 > Loading wavs from: ./500samples_30m/wavs
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.1
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:./TTS/hifigan-v2/scale_stats_new.npy
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Generator Model: hifigan_generator
 > Discriminator Model: hifigan_discriminator
 > Restoring from model_file.pth.tar...
 > Restoring Generator Model...
 > Restoring Generator Optimizer...
 > 